## 텍스트 데이터 다루기

### 노이즈 유형

1) 문장부호
- 문장부호를 찾고 양쪽에 공백을 추가하는 방법
  - 하지만 "45,756" 이 "45 , 756" 이렇게 바뀔 수 있겠지. -> 이런 경우는 불가피한 손실로 취급하고 넘어간다.
  

In [2]:
def pad_punctuation(sentence, punc):
    for p in punc:
        sentence = sentence.replace(p, " " + p + " ")

    return sentence

sentence = "Hi, my name is john."

print(pad_punctuation(sentence, [".", "?", "!", ","]))

Hi ,  my name is john . 


2) 대소문자
- 모든 단어를 대문자나 소문자로 일괄 변경

In [3]:
sentence = "First, open the first chapter."

print(sentence.lower())

print(sentence.upper())

first, open the first chapter.
FIRST, OPEN THE FIRST CHAPTER.


3) 특수문자
- 몇 몇 특수문자 기호들을 정의해 이를 제외하곤 모두 제거

In [5]:
import re

sentence = "He is a ten-year-old boy."
sentence = re.sub("([^a-zA-Z.,?!])", " ", sentence)

print(sentence)

He is a ten year old boy.


### 문장을 정제하는 함수 정의

In [8]:
# From The Project Gutenberg
# (https://www.gutenberg.org/files/2397/2397-h/2397-h.htm)

corpus = \
"""
In the days that followed I learned to spell in this uncomprehending way a great many words, among them pin, hat, cup and a few verbs like sit, stand and walk. 
But my teacher had been with me several weeks before I understood that everything has a name.
One day, we walked down the path to the well-house, attracted by the fragrance of the honeysuckle with which it was covered. 
Some one was drawing water and my teacher placed my hand under the spout. 
As the cool stream gushed over one hand she spelled into the other the word water, first slowly, then rapidly. 
I stood still, my whole attention fixed upon the motions of her fingers. 
Suddenly I felt a misty consciousness as of something forgotten—a thrill of returning thought; and somehow the mystery of language was revealed to me. 
I knew then that "w-a-t-e-r" meant the wonderful cool something that was flowing over my hand. 
That living word awakened my soul, gave it light, hope, joy, set it free! 
There were barriers still, it is true, but barriers that could in time be swept away.
""" 

def cleaning_text(text, punc, regex):
    # 노이즈 유형 (1) 문장부호 공백추가
    for p in punc:
        text = text.replace(p, " " + p + " ")

    # 노이즈 유형 (2), (3) 소문자화 및 특수문자 제거
    text = re.sub(regex, " ", text).lower()

    return text

print(cleaning_text(corpus, [".", ",", "!", "?"], "([^a-zA-Z0-9.,?!\n])"))


in the days that followed i learned to spell in this uncomprehending way a great many words ,  among them pin ,  hat ,  cup and a few verbs like sit ,  stand and walk .  
but my teacher had been with me several weeks before i understood that everything has a name . 
one day ,  we walked down the path to the well house ,  attracted by the fragrance of the honeysuckle with which it was covered .  
some one was drawing water and my teacher placed my hand under the spout .  
as the cool stream gushed over one hand she spelled into the other the word water ,  first slowly ,  then rapidly .  
i stood still ,  my whole attention fixed upon the motions of her fingers .  
suddenly i felt a misty consciousness as of something forgotten a thrill of returning thought  and somehow the mystery of language was revealed to me .  
i knew then that  w a t e r  meant the wonderful cool something that was flowing over my hand .  
that living word awakened my soul ,  gave it light ,  hope ,  joy ,  set it

### 단어의 희소 표현과 분산 표현

{  
    //     [성별, 연령]  
    남자: [-1.0, 0.0], // 이를테면 0.0 이 "관계없음 또는 중립적" 을 의미할 수 있겠죠!  
    여자: [1.0, 0.0],  
    소년: [-1.0, -0.7],  
    소녀: [1.0, -0.7],  
    할머니: [1.0, 0.7],  
    할아버지: [-1.0, 0.7],  
    아저씨: [-1.0, 0.2],  
    아줌마: [1.0, 0.2]  
}

{  
    //      [성별, 연령, 과일, 색깔]  
    남자: [-1.0, 0.0, 0.0, 0.0],  
    여자: [1.0, 0.0, 0.0, 0.0],  
    사과: [0.0, 0.0, 1.0, 0.5],   // 빨갛게 잘 익은 사과  
    바나나: [0.0, 0.0, 1.0, -0.5] // 노랗게 잘 익은 바나나  
}  

두 고차원 벡터의 유사도는 코사인 유사도(Cosine Similarity) 를 통해 구할 수 있다.

우린 Embedding 레이어를 사용해 각 단어가 몇 차원의 속성을 가질지 정의하는 방식으로

단어의 분산 표현(distributed representation) 를 구현하는 방식을 주로 사용하게 된다.

만약 100개의 단어를 256차원의 속성으로 표현하고 싶다면 Embedding 레이어는 아래와 같이 정의된다.

embedding_layer = tf.keras.layers.Embedding(input_dim=100, output_dim=256)

위 단어의 분산 표현에는 우리가 일일이 정의할 수 없는 어떤 추상적인 속성들이 256차원 안에 골고루 분산되어 표현된다.

희소 표현처럼 속성값을 임의로 지정해 주는 것이 아니라, 수많은 텍스트 데이터를 읽어가며 적합한 값을 찾아갈 것이다.

적절히 훈련된 분산 표현 모델을 통해 우리는 단어 간의 의미 유사도를 계산하거나, 이를 feature로 삼아 복잡한 자연어처리 모델을 훈련시킬 수 있게 된다.

### 토큰화

한 문장에서 단어의 수는 어떻게 정의할 수 있을까?

"그녀는 나와 밥을 먹는다" 를
- "그녀는" "나와" "밥을" "먹는다" 로 할 수 도 있고, 
- "그녀" "는" "나" "와" "밥" "을" "먹는다" 
로도 잘게 쪼갤 수 있다.

이게, 우리가 정의할 토큰화 기법이 결정할 부분이다!

#### 공백 기반 토큰화

In [9]:
corpus = \
"""
in the days that followed i learned to spell in this uncomprehending way a great many words ,  among them pin ,  hat ,  cup and a few verbs like sit ,  stand and walk .  
but my teacher had been with me several weeks before i understood that everything has a name . 
one day ,  we walked down the path to the well house ,  attracted by the fragrance of the honeysuckle with which it was covered .  
some one was drawing water and my teacher placed my hand under the spout .  
as the cool stream gushed over one hand she spelled into the other the word water ,  first slowly ,  then rapidly .  
i stood still ,  my whole attention fixed upon the motions of her fingers .  
suddenly i felt a misty consciousness as of something forgotten a thrill of returning thought  and somehow the mystery of language was revealed to me .  
i knew then that  w a t e r  meant the wonderful cool something that was flowing over my hand .  
that living word awakened my soul ,  gave it light ,  hope ,  joy ,  set it free !  
there were barriers still ,  it is true ,  but barriers that could in time be swept away . 
"""

tokens = corpus.split()

print("문장이 포함하는 Tokens:", tokens)

문장이 포함하는 Tokens: ['in', 'the', 'days', 'that', 'followed', 'i', 'learned', 'to', 'spell', 'in', 'this', 'uncomprehending', 'way', 'a', 'great', 'many', 'words', ',', 'among', 'them', 'pin', ',', 'hat', ',', 'cup', 'and', 'a', 'few', 'verbs', 'like', 'sit', ',', 'stand', 'and', 'walk', '.', 'but', 'my', 'teacher', 'had', 'been', 'with', 'me', 'several', 'weeks', 'before', 'i', 'understood', 'that', 'everything', 'has', 'a', 'name', '.', 'one', 'day', ',', 'we', 'walked', 'down', 'the', 'path', 'to', 'the', 'well', 'house', ',', 'attracted', 'by', 'the', 'fragrance', 'of', 'the', 'honeysuckle', 'with', 'which', 'it', 'was', 'covered', '.', 'some', 'one', 'was', 'drawing', 'water', 'and', 'my', 'teacher', 'placed', 'my', 'hand', 'under', 'the', 'spout', '.', 'as', 'the', 'cool', 'stream', 'gushed', 'over', 'one', 'hand', 'she', 'spelled', 'into', 'the', 'other', 'the', 'word', 'water', ',', 'first', 'slowly', ',', 'then', 'rapidly', '.', 'i', 'stood', 'still', ',', 'my', 'whole', 'attenti

#### 형태소 기반 토큰화

한국어 형태소 분석기에서 대표적인 KoNLPy를 사용해보자.  
https://konlpy-ko.readthedocs.io/ko/v0.4.3/

한국어 형태소 분석기 성능 비교  
https://iostream.tistory.com/m/144

In [1]:
from konlpy.tag import Hannanum,Kkma,Komoran,Mecab,Okt

ModuleNotFoundError: No module named 'konlpy'

In [2]:
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     |████████████████████████████████| 19.4 MB 28.3 MB/s eta 0:00:01
     |████████████████████████████████| 453 kB 82.0 MB/s eta 0:00:01


In [1]:
from konlpy.tag import Hannanum,Kkma,Komoran,Mecab,Okt

In [2]:
tokenizer_list = [Hannanum(),Kkma(),Komoran(),Mecab(),Okt()]

kor_text = '코로나바이러스는 2019년 12월 중국 우한에서 처음 발생한 뒤 전 세계로 확산된, 새로운 유형의 호흡기 감염 질환입니다.'

for tokenizer in tokenizer_list:
    print('[{}] \n{}'.format(tokenizer.__class__.__name__, tokenizer.pos(kor_text)))

[Hannanum] 
[('코로나바이러스', 'N'), ('는', 'J'), ('2019년', 'N'), ('12월', 'N'), ('중국', 'N'), ('우한', 'N'), ('에서', 'J'), ('처음', 'M'), ('발생', 'N'), ('하', 'X'), ('ㄴ', 'E'), ('뒤', 'N'), ('전', 'N'), ('세계', 'N'), ('로', 'J'), ('확산', 'N'), ('되', 'X'), ('ㄴ', 'E'), (',', 'S'), ('새롭', 'P'), ('은', 'E'), ('유형', 'N'), ('의', 'J'), ('호흡기', 'N'), ('감염', 'N'), ('질환', 'N'), ('이', 'J'), ('ㅂ니다', 'E'), ('.', 'S')]
[Kkma] 
[('코로나', 'NNG'), ('바', 'NNG'), ('이러', 'MAG'), ('슬', 'VV'), ('는', 'ETD'), ('2019', 'NR'), ('년', 'NNM'), ('12', 'NR'), ('월', 'NNM'), ('중국', 'NNG'), ('우', 'NNG'), ('하', 'XSV'), ('ㄴ', 'ETD'), ('에', 'VV'), ('서', 'ECD'), ('처음', 'NNG'), ('발생', 'NNG'), ('하', 'XSV'), ('ㄴ', 'ETD'), ('뒤', 'NNG'), ('전', 'NNG'), ('세계', 'NNG'), ('로', 'JKM'), ('확산', 'NNG'), ('되', 'XSV'), ('ㄴ', 'ETD'), (',', 'SP'), ('새', 'NNG'), ('롭', 'XSA'), ('ㄴ', 'ETD'), ('유형', 'NNG'), ('의', 'JKG'), ('호흡기', 'NNG'), ('감염', 'NNG'), ('질환', 'NNG'), ('이', 'VCP'), ('ㅂ니다', 'EFN'), ('.', 'SF')]
[Komoran] 
[('코로나바이러스', 'NNP'), ('는', 'JX'), ('2019', 'SN'

### 트러블슈팅 (mecab 설치)

bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh) 이걸로 설치하지만 잘 안되는 경우가 있다.

그럴 땐 하나씩 하나씩 수동으로 설치한다.

https://i-am-eden.tistory.com/m/9 참고

모두 다 설치한 후 커널을 재시작하면 된다.

In [10]:
!bash _etc/mecab.sh

mecab-ko is already installed
mecab-ko-dic is already installed
mecab-python is already installed
Done.


In [11]:
import MeCab
m = MeCab.Tagger()
out= m.parse("미캅이 잘 설치되었는지 확인중입니다.")
print(out)

미	NNP,지명,F,미,*,*,*,*
캅	NNP,인명,T,캅,*,*,*,*
이	JKS,*,F,이,*,*,*,*
잘	MAG,*,T,잘,*,*,*,*
설치	NNG,*,F,설치,*,*,*,*
되	XSV,*,F,되,*,*,*,*
었	EP,*,T,었,*,*,*,*
는지	EC,*,F,는지,*,*,*,*
확인	NNG,*,T,확인,*,*,*,*
중	NNB,*,T,중,*,*,*,*
입니다	VCP+EF,*,F,입니다,Inflect,VCP,EF,이/VCP/*+ᄇ니다/EF/*
.	SF,*,*,*,*,*,*,*
EOS



### Byte Pair Encoding(BPE)

가장 많이 등장하는 바이트 쌍(Byte Pair) 을 새로운 단어로 치환하여 압축하는 작업을 반복하는 방식으로 동작

만약 수많은 데이터를 사용해 만든 BPE 사전으로 모델을 학습시키고 문장을 생성하게 했다고 합시다.  
그게 [i, am, a, b, o, y, a, n, d, you, are, a, gir, l]이라면, 어떤 기준으로 이들을 결합해서 문장을 복원하죠?  
몽땅 한꺼번에 합쳤다간 끔찍한 일이 벌어질 것만 같습니다...

In [ ]:
aaabdaaabac # 가장 많이 등장한 바이트 쌍 "aa"를 "Z"로 치환합니다.
→ 
ZabdZabac   # "aa" 총 두 개가 치환되어 4바이트를 2바이트로 압축하였습니다.
Z=aa        # 그다음 많이 등장한 바이트 쌍 "ab"를 "Y"로 치환합니다.
→ 
ZYdZYac     # "ab" 총 두 개가 치환되어 4바이트를 2바이트로 압축하였습니다.
Z=aa        # 여기서 작업을 멈추어도 되지만, 치환된 바이트에 대해서도 진행한다면
Y=ab        # 가장 많이 등장한 바이트 쌍 "ZY"를 "X"로 치환합니다.
→ 
XdXac
Z=aa
Y=ab
X=ZY       # 압축이 완료되었습니다!

In [17]:
import re, collections

# 임의의 데이터에 포함된 단어들입니다.
# 우측의 정수는 임의의 데이터에 해당 단어가 포함된 빈도수입니다.
vocab = {
    'l o w '      : 5,
    'l o w e r '  : 2,
    'n e w e s t ': 6,
    'w i d e s t ': 3
}

num_merges = 5

def get_stats(vocab):
    """
    단어 사전을 불러와
    단어는 공백 단위로 쪼개어 문자 list를 만들고
    빈도수와 쌍을 이루게 합니다. (symbols)
    """
    pairs = collections.defaultdict(int)
    
    for word, freq in vocab.items():
        symbols = word.split()

        for i in range(len(symbols) - 1):             # 모든 symbols를 확인하여 
            pairs[symbols[i], symbols[i + 1]] += freq  # 문자 쌍의 빈도수를 저장합니다. 
        
    return pairs

def merge_vocab(pair, v_in):
    v_out = {}
    bigram = re.escape(' '.join(pair))
    p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
    
    for word in v_in:
        w_out = p.sub(''.join(pair), word)
        v_out[w_out] = v_in[word]
        
    return v_out, pair[0] + pair[1]

token_vocab = []

for i in range(num_merges):
    print(">> Step {0}".format(i + 1))
    
    pairs = get_stats(vocab)
    best = max(pairs, key=pairs.get)  # 가장 많은 빈도수를 가진 문자 쌍을 반환합니다.
    vocab, merge_tok = merge_vocab(best, vocab)
    print("다음 문자 쌍을 치환:", merge_tok)
    print("변환된 Vocab:\n", vocab, "\n")
    
    token_vocab.append(merge_tok)
    
print("Merge Vocab:", token_vocab)

>> Step 1
다음 문자 쌍을 치환: es
변환된 Vocab:
 {'l o w ': 5, 'l o w e r ': 2, 'n e w es t ': 6, 'w i d es t ': 3} 

>> Step 2
다음 문자 쌍을 치환: est
변환된 Vocab:
 {'l o w ': 5, 'l o w e r ': 2, 'n e w est ': 6, 'w i d est ': 3} 

>> Step 3
다음 문자 쌍을 치환: lo
변환된 Vocab:
 {'lo w ': 5, 'lo w e r ': 2, 'n e w est ': 6, 'w i d est ': 3} 

>> Step 4
다음 문자 쌍을 치환: low
변환된 Vocab:
 {'low ': 5, 'low e r ': 2, 'n e w est ': 6, 'w i d est ': 3} 

>> Step 5
다음 문자 쌍을 치환: ne
변환된 Vocab:
 {'low ': 5, 'low e r ': 2, 'ne w est ': 6, 'w i d est ': 3} 

Merge Vocab: ['es', 'est', 'lo', 'low', 'ne']


### Wordpiece Model(WPM)

이에 구글에서 BPE를 변형해 제안한 알고리즘이 바로 WPM입니다. WPM은 BPE에 대해 두 가지 차별성을 가집니다.
- 공백 복원을 위해 단어의 시작 부분에 언더바 _ 를 추가합니다.
- 빈도수 기반이 아닌 가능도(Likelihood)를 증가시키는 방향으로 문자 쌍을 합칩니다.

즉 [_i, _am, _a, _b, o, y, _a, n, d, _you, _are, _a, _gir, l]로 토큰화를 한다는 것입니다.  
이렇게 하면 문장을 복원하는 과정이 1) 모든 토큰을 합친 후, 2) 언더바 _를 공백으로 치환으로 마무리되어 간편하죠.

### soynlp

이외에도 한국어를 위한 토크나이저로 soynlp를 활용할 수 있습니다.  
soynlp는 한국어 자연어 처리를 위한 라이브러리인데요.  
토크나이저 외에도 단어 추출, 품사 판별, 전처리 기능도 제공합니다.

형태소 기반의 토크나이저가 미등록 단어에 취약하기 때문에 WordPiece Model을 사용하는 것처럼,  
형태소 기반인 koNLPy의 단점을 해결하기 위해 soynlp를 사용할 수 있습니다.

### 토큰에게 의미를 부여하기

#### Word2Vec

Word2Vec은 "단어를 벡터로 만든다"는 멋진 이름을 가지고 있습니다.  

난 오늘 술을 한 잔 마셨어 라는 문장의 각 단어 즉, *동시에 등장하는 단어끼리는 연관성이 있다*는 아이디어로 시작된 알고리즘입니다.

예문의 경우 다른 단어는 몰라도 술과 마셨어는 괜찮은 연관성을 캐치해낼 수 있겠네요.

https://wikidocs.net/22660

#### FastText

Word2Vec은 정말 좋은 방법이지만, 연산의 빈부격차가 존재했습니다. 

자주 등장하지 않는 단어는 최악의 경우 단 한 번의 연산만을 거쳐 랜덤하게 초기화된 값과 크게 다르지 않은 상태로 알고리즘이 종료될 수 있습니다. FastText는 이를 해결하기 위해 BPE와 비슷한 아이디어를 적용했습니다.

한국어를 위한 어휘 임베딩의 개발 -1-  
https://brunch.co.kr/@learning/7

#### ELMo - the 1st Contextualized Word Embedding

위에 소개했던 Word Embedding 알고리즘들은 (역시나) 정말 훌륭하지만, 여전히 고질적인 문제점이 있습니다. 

바로 고정적이라는 겁니다! 무슨 말이냐면, 동음이의어를 처리할 수 없다는 얘기입니다.

- 이렇게나 탐스럽고 먹음직스러운 사과를 보셨나요?
- 저의 간절한 사과를 받아주시기 바랍니다.

우리는 이 두 문장에 나오는 '사과'의 의미가 다르다는 것을 알고 있습니다.  
그러나 Word2Vec이든 FastText이든 간에 이 두 문장에 나오는 사과의 워드 벡터값은 동일할 수밖에 없습니다.

Context-sensitive Grammar를 따르는 자연어를 이해하려면 문맥(context)의 활용이 필수적입니다.
- 여기서 '사과'의 context가 되는 것은 무엇일까요? 
  - 첫 문장이라면 탐스럽고 먹음직스러운 이 될 것이고 다음 문장이라면 간절한 이 될 것입니다. 
- 즉, 단어의 의미 벡터를 구하기 위해서는 그 단어만 필요한 것이 아니라 그 단어가 놓인 주변 단어 배치의 맥락이 함께 고려되는 Word Embedding이 필요한 것입니다. 

이런 개념을 Contextualized Word Embedding이라고 합니다.

전이 학습 기반 NLP (1): ELMo  
https://brunch.co.kr/@learning/12

## 마무리

이번에 배운 내용은 토큰화와 분산 표현이 중심입니다. 

문장이 입력되면 적절히 토큰화를 하고 토큰을 임베딩(Embedding)을 통해 분산 표현으로 만드는 것이지요.  
분산 표현은 벡터이므로 이제 인공지능에 활용할 수 있습니다.

토큰화에 사용되는 방법은 언어마다 다른데요. 문장 구성 성분이 다르기 때문입니다.  
조사가 있는 한국어는 형태소 기반인 koNLPy를 주로 쓰고, WordPiece Model인 SentencePiece를 쓸 수도 있어요. 물론 그 외에 다른 방법도 있습니다.

토큰화를 마친 후 임베딩을 할 때는 토큰마다 독립적으로 만들지 않고 토큰 간의 관계성을 주입합니다.  
그래야 문장을 구성할 때 적절히 사용될 수 있기 때문이에요.  
이렇게 토큰 간의 관계성을 고려해서 만든 것으로는 Word2Vec, FastText 등이 있어요. 거기다가 문장의 문맥까지 고려하는 ELMo까지 등장했습니다.